In [26]:
import os,sys
sys.path.append(os.path.abspath('./../../'))
from Controllers.MarkedCellController import MarkedCellController,CellSources
from Controllers.SqlController import SqlController
from model.annotation_points import MarkedCellView
from sqlalchemy import inspect
inst = inspect(MarkedCellView)
attr_names = [c_attr.key for c_attr in inst.mapper.column_attrs]
import pandas as pd
from cell_extractor.DetectorMetrics import AnnotationProximityTool
from cell_extractor.CellDetector import CellDetector
import numpy as np
from collections import Counter
user_look_up = {38:'Marissa',41:'Julian',2:'Beth',3:'Hannah'}

def get_DataFrame_from_query_result(results,category,factor):
    values = []
    for i in results:
        source = i.source.value
        if '-' in source:
            source = source.split('-')[1]    
        x,y,z = np.array([i.x,i.y,i.z]).astype(float)/factor
        values.append([x,y,z,f'{category}_{user_look_up[i.FK_annotator_id]}_{source}'])
    # values = [[eval(f'j.{i}')  for j in results ] for i in attr_names]
    df = pd.DataFrame(dict(zip(['x','y','section','name'],np.array(values).T)))
    df["x"] = pd.to_numeric(df["x"])
    df["y"] = pd.to_numeric(df["y"])
    df["section"] = pd.to_numeric(df["section"])
    return df

def get_DataFrame_from_detection_df(df,label):
    data = np.array([df.col.to_numpy().astype(int),df.row.to_numpy().astype(int),\
        df.section.to_numpy().astype(int)])
    data = pd.DataFrame(data.T,columns=['x','y','section'])
    data['name'] = [label]*len(df)
    return data

def print_human_machine_overlap(animal,annotator_id = 3):
    controller = SqlController()
    factor = controller.get_resolution(animal)
    controller = MarkedCellController()
    search_dict = {'FK_prep_id':animal,'FK_cell_type_id':1,'FK_annotator_id':annotator_id}
    cells = controller.get_marked_cells(search_dictionary=search_dict)
    cells = get_DataFrame_from_query_result(cells,'Round1',factor) 
    detector = CellDetector(animal,round=2)
    detections = detector.load_detections()
    sures = detections[detections.predictions==2]
    unsures = detections[detections.predictions==0]
    sures = get_DataFrame_from_detection_df(sures,'sures')
    unsures = get_DataFrame_from_detection_df(unsures,'unsures')
    tool = AnnotationProximityTool()
    tool.pair_distance=30
    tool.set_annotations_to_compare(pd.concat([sures,unsures,cells]))
    tool.find_equivalent_points()
    print(f'total sure {len(sures)} total unsure {len(unsures)}')
    for i in Counter([tuple(i) for i in tool.pair_categories.values()]).most_common():
        print(i)

In [27]:
print_human_machine_overlap('DK39',3)

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 13919
after removing duplicates 13885
total sure 8922 total unsure 2849
(('sures',), 8894)
(('unsures',), 2830)
(('Round1_Hannah_POSITIVE',), 2129)
(('sures', 'Round1_Hannah_POSITIVE'), 12)
(('sures', 'sures'), 6)
(('unsures', 'Round1_Hannah_POSITIVE'), 5)
(('sures', 'unsures'), 4)
(('unsures', 'unsures', 'unsures'), 2)
(('unsures', 'unsures'), 2)
(('Round1_Hannah_POSITIVE', 'Round1_Hannah_POSITIVE'), 1)


In [28]:
print_human_machine_overlap('DK52',2)

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 13575
after removing duplicates 11971
total sure 9488 total unsure 2137
(('sures',), 7980)
(('unsures',), 2052)
(('sures', 'Round1_Beth_POSITIVE'), 1490)
(('Round1_Beth_POSITIVE',), 352)
(('unsures', 'Round1_Beth_POSITIVE'), 75)
(('sures', 'Round1_Beth_POSITIVE', 'Round1_Beth_POSITIVE'), 13)
(('unsures', 'Round1_Beth_POSITIVE', 'Round1_Beth_POSITIVE'), 3)
(('sures', 'sures'), 2)
(('unsures', 'unsures'), 2)
(('sures', 'unsures'), 1)
(('unsures', 'unsures', 'Round1_Beth_POSITIVE'), 1)


In [29]:
animal = 'DK39'
controller = SqlController()
factor = controller.get_resolution(animal)
controller = MarkedCellController()
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':1,'FK_annotator_id':3}
cells = controller.get_marked_cells(search_dictionary=search_dict)
cells = get_DataFrame_from_query_result(cells,'Round1',factor) 
detector = CellDetector(animal,round=2)
detections = detector.load_detections()
sures = detections[detections.predictions==2]
unsures = detections[detections.predictions==0]
sures = get_DataFrame_from_detection_df(sures,'sures')
unsures = get_DataFrame_from_detection_df(unsures,'unsures')


In [31]:
cells.x.max(),cells.y.max(),sures.x.max(),sures.y.max()

(52145.84615384616, 26042.0, 53330, 27245)

In [ ]:
tool = AnnotationProximityTool()
tool.pair_distance=30
tool.set_annotations_to_compare(pd.concat([sures,unsures,cells]))
tool.find_equivalent_points()
print(f'total sure {len(sures)} total unsure {len(unsures)}')
for i in Counter([tuple(i) for i in tool.pair_categories.values()]).most_common():
    print(i)